# Using FFT to do convolution.
[Source code link from StackOverflow](https://stackoverflow.com/questions/40703751/using-fourier-transforms-to-do-convolution?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)

In [7]:
import sys
from scipy import signal

from scipy import linalg
import numpy as np

x = [[1 , 0 , 0 , 0] , [0 , -1 , 0 , 0] , [0 , 0 , 3 , 0] , [0 , 0 , 0 , 1]]
x = np.array(x)
y = [[4 , 5] , [3 , 4]]
y = np.array(y)

standard_conv = signal.convolve2d(x , y , 'full')

print("conv:" , standard_conv)

s1 = np.array(x.shape)
s2 = np.array(y.shape)

size = s1 + s2 - 1


fsize = 2 ** np.ceil(np.log2(size)).astype(int)
fslice = tuple([slice(0, int(sz)) for sz in size])

# Along each axis, if the given shape (fsize) is smaller than that of the input, the input is cropped. 
# If it is larger, the input is padded with zeros. if s is not given, the shape of the input along the axes 
# specified by axes is used.
new_x = np.fft.fft2(x, fsize)

new_y = np.fft.fft2(y, fsize)
result = np.fft.ifft2(new_x*new_y)[fslice].copy()
result_int = np.array(result.real , np.int32)

my_result = np.array(result, np.double)
print("my_result (doubles): ", my_result)

print("fft for my method (ints):" , result_int)
print("is my method correct (for ints): ", np.array_equal(result_int, standard_conv))
print("fft for my method (doubles):" , result)

print("fft with int32 output:" , np.array(signal.fftconvolve(x ,y) , np.int32))
lib_result = np.array(signal.fftconvolve(x, y) , np.double)
print("fft with double output:" , np.allclose(my_result, lib_result, atol=1e-12))

# the correct way is to take the amplitude:  the abs of a complex number gives us its amplitude/mangnitude
lib_magnitude = np.abs(signal.fftconvolve(x, y))
print("lib_magnitude: ", lib_magnitude)
my_magnitude = np.abs(result)
print("is the magnitude correct: ", np.allclose(my_magnitude, lib_magnitude, atol=1e-12))

conv: [[ 4  5  0  0  0]
 [ 3  0 -5  0  0]
 [ 0 -3  8 15  0]
 [ 0  0  9 16  5]
 [ 0  0  0  3  4]]
my_result (doubles):  [[ 4.00000000e+00  5.00000000e+00  2.22044605e-16  1.11022302e-15
   0.00000000e+00]
 [ 3.00000000e+00  2.15309973e-16 -5.00000000e+00 -1.77635684e-15
  -1.11022302e-15]
 [-2.77555756e-16 -3.00000000e+00  8.00000000e+00  1.50000000e+01
   5.55111512e-17]
 [ 4.99600361e-16 -3.08515238e-15  9.00000000e+00  1.60000000e+01
   5.00000000e+00]
 [ 0.00000000e+00 -4.44089210e-16 -2.22044605e-16  3.00000000e+00
   4.00000000e+00]]
fft for my method (ints): [[ 4  5  0  0  0]
 [ 3  0 -5  0  0]
 [ 0 -3  8 15  0]
 [ 0  0  9 16  5]
 [ 0  0  0  3  4]]
is my method correct (for ints):  True
fft for my method (doubles): [[ 4.00000000e+00-6.10622664e-16j  5.00000000e+00+1.83758918e-15j
   2.22044605e-16-1.38777878e-17j  1.11022302e-15+1.61104109e-15j
   0.00000000e+00+3.49720253e-15j]
 [ 3.00000000e+00+1.11980992e-15j  2.15309973e-16-8.70143447e-16j
  -5.00000000e+00-3.10247810e-15j -1.

/home/adam/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:34: ComplexWarning: Casting complex values to real discards the imaginary part
